# **Transverse Field Ising Model (TFIM) Analysis**

In [33]:
import numpy as np
from typing import Sequence

from quri_parts.core.state import quantum_state, apply_circuit, ParametricCircuitQuantumState
from quri_parts.circuit import LinearMappedUnboundParametricQuantumCircuit, CONST
from quri_parts.algo.ansatz import Z2SymmetryPreservingReal
from quri_parts.core.operator import Operator, pauli_label, PAULI_IDENTITY
from quri_parts.circuit.utils.circuit_drawer import draw_circuit

from quri_parts.core.estimator import Estimatable
from quri_parts.core.estimator.gradient import create_numerical_gradient_estimator
from quri_parts.qulacs.estimator import create_qulacs_vector_concurrent_parametric_estimator, create_qulacs_vector_parametric_estimator
from quri_parts.algo.optimizer import Adam, OptimizerStatus, Optimizer, OptimizerState, CostFunction, GradientFunction
from quri_parts.circuit.parameter_shift import ShiftedParameters

## **1. Model Definition: Hamiltonian and Spin Algebra**

The 1D TFIM Hamiltonian is:

$$H = -J\sum_{i=1}^{N-1} Z_i Z_{i+1} - h\sum_{i=1}^N X_i$$

- **Z:** Pauli-Z operator (spin measurement)
- **X:** Pauli-X operator (transverse field)
- **Ferromagnetic coupling (\(J\) term)**: Favors alignment of neighboring spins along \(z\).  
- **Transverse field (\(h\) term)**: Induces quantum fluctuations, polarizing spins along \(x\).  

**Spin Operators**:  
- $ \sigma_n^z = \begin{pmatrix}1&0\\0&-1\end{pmatrix}), (\sigma_n^x = \begin{pmatrix}0&1\\1&0\end{pmatrix})$ act on the (n)-th spin.

## **2. Ground States and Symmetry**

### **Ferromagnetic Phase (J > h)**  
Two degenerate ground states:
$$
|\psi^{0+}\rangle = \prod_{n=1}^N | \uparrow \rangle_n, \quad |\psi^{0-}\rangle = \prod_{n=1}^N | \downarrow \rangle_n
$$  
- These break the $\mathbf{Z}_2$ symmetry $(σ_n^z→−σ_n^z)$ and exhibit non-zero magnetization $M =\pm N$


### **Paramagnetic Phase (h > J)**
A unique symmetric ground state polarized along x:
$$
|\psi^{0\to}\rangle = \prod_{n=1}^N \frac{1}{\sqrt{2}} \left( | \uparrow \rangle_n + | \downarrow \rangle_n \right) = \prod_{n=1}^N | \rightarrow \rangle_n
$$  
Here $\mathbf{Z}_2$ symmetry is preserved and M = 0.

## **3. Quantum Phase Transition at \(h = J\)**

### **Exact Solution via Jordan-Wigner Transformation**  
Map spins to fermions:  
$$
a_n = \sigma_n^- \prod_{m=1}^{n-1} \sigma_m^x, \quad a_n^\dagger = \sigma_n^+ \prod_{m=1}^{n-1} \sigma_m^x
$$  
**Hamiltonian in Fermionic Terms**:  
$$
H = J \sum_{n=1}^{N-1} (a_n^\dagger - a_n)(a_{n+1}^\dagger + a_{n+1}) + h \sum_{n=1}^N (1 - 2a_n^\dagger a_n)
$$  
Diagonalize using Fourier and Bogoliubov transformations. The energy gap:  
$$
\Delta = 2|J - h|
$$  
**Critical Point (\(h = J\))**:  
$(\Delta \to 0)$, leading to gapless excitations.  

### **Duality (Kramers-Wannier)**  
Under \(J \leftrightarrow h\), the Hamiltonian retains its form, confirming criticality at \(h = J\).  

**Critical Exponents**:  
From the exact solution, exponents match the 2D classical Ising model:  
$$
\nu = 1, \quad \beta = \frac{1}{8}, \quad \gamma = \frac{7}{4}
$$  

# **QURI Parts Implementation**

In [83]:
parametric_circuit = Z2SymmetryPreservingReal(5, 7)
draw_circuit(parametric_circuit)

                           ___     ___     ___                             ___  
                          |PR |   |PRZ|   |PR |                           |PR | 
--------------------------|8  |---|9  |---|11 |---------------------------|24 |-
                          |   |   |___|   |   |                           |   | 
   ___     ___     ___    |   |    ___    |   |    ___     ___     ___    |   | 
  |PR |   |PRZ|   |PR |   |   |   |PRZ|   |   |   |PR |   |PRZ|   |PR |   |   | 
--|0  |---|1  |---|3  |---|   |---|10 |---|   |---|16 |---|17 |---|19 |---|   |-
  |   |   |___|   |   |   |___|   |___|   |___|   |   |   |___|   |   |   |___| 
  |   |    ___    |   |    ___     ___     ___    |   |    ___    |   |    ___  
  |   |   |PRZ|   |   |   |PR |   |PRZ|   |PR |   |   |   |PRZ|   |   |   |PR | 
--|   |---|2  |---|   |---|12 |---|13 |---|15 |---|   |---|18 |---|   |---|28 |-
  |___|   |___|   |___|   |   |   |___|   |   |   |___|   |___|   |___|   |   | 
   ___     ___     ___    | 

In [84]:
def get_raw_param_state_and_shifted_parameters(
    state,
    params: Sequence[float]
):

    param_mapping = state.parametric_circuit.param_mapping
    raw_circuit = state.parametric_circuit.primitive_circuit()
    raw_param_state = ParametricCircuitQuantumState(state.qubit_count, raw_circuit)
    
    parameter_shift = ShiftedParameters(param_mapping)
    derivatives = parameter_shift.get_derivatives()
    shifted_parameters = [
        d.get_shifted_parameters_and_coef(params) for d in derivatives
    ]

    return raw_param_state, shifted_parameters

In [85]:
def get_parameter_shift_gradient(
    op: Estimatable,
    raw_state: ParametricCircuitQuantumState,
    shifted_params_and_coefs
) -> list[complex]:
    
    # Collect gate parameters to be evaluated
    gate_params = set()
    for params_and_coefs in shifted_params_and_coefs:
        for p, _ in params_and_coefs:
            gate_params.add(p)
    gate_params_list = list(gate_params)

    # Prepare a parametric estimator
    estimator = create_qulacs_vector_concurrent_parametric_estimator()

    # Estimate the expectation values
    estimates = estimator(op, raw_state, gate_params_list)
    estimates_dict = dict(zip(gate_params_list, estimates))

    # Sum up the expectation values with the coefficients multiplied
    gradient = []
    for params_and_coefs in shifted_params_and_coefs:
        g = 0.0
        for p, c in params_and_coefs:
            g += estimates_dict[p].value * c
        gradient.append(g)

    return gradient

In [ ]:
gradient = get_parameter_shift_gradient(hamiltonian, raw_state, shifted_params_and_coefs)

In [86]:
estimator = create_qulacs_vector_parametric_estimator()

def cost_fn(param_values: Sequence[float]) -> float:
    estimate = estimator(hamiltonian, parametric_state, param_values)
    return estimate.value.real

In [99]:
adam_optimizer = Adam()

cb_state = quantum_state(5, bits=0b00011)
parametric_state = apply_circuit(parametric_circuit, cb_state)

In [100]:
qulacs_concurrent_parametric_estimator = create_qulacs_vector_concurrent_parametric_estimator()
gradient_estimator = create_numerical_gradient_estimator(
    qulacs_concurrent_parametric_estimator,
    delta=1e-4,
)

def grad_fn(param_values: Sequence[float]) -> Sequence[float]:
    estimate = gradient_estimator(hamiltonian, parametric_state, param_values)
    return np.asarray([g.real for g in estimate.values])

In [101]:
def vqe(
    init_params: Sequence[float],
    cost_fn,
    grad_fn,
    optimizer
):
    opt_state = optimizer.get_init_state(init_params)
    while True:
        opt_state = optimizer.step(opt_state, cost_fn, grad_fn)
        if opt_state.status == OptimizerStatus.FAILED:
            print("Optimizer failed")
            break
        if opt_state.status == OptimizerStatus.CONVERGED:
            print("Optimizer converged")
            break
    return opt_state

In [102]:
param_state = quantum_state(5, circuit=parametric_circuit)
raw_state, shifted_params_and_coefs = get_raw_param_state_and_shifted_parameters(
    param_state, [0] * parametric_circuit.parameter_count
)

In [103]:
hamiltonian = Operator({
    pauli_label("Z0"): -1,
    pauli_label("Z1"): -1,
    pauli_label("Z2"): -1,
    pauli_label("Z3"): -1,
    pauli_label("Z4"): -1,
})

In [104]:
init_params = [0.1] * parametric_circuit.parameter_count
result = vqe(init_params, cost_fn, grad_fn, adam_optimizer)
print("Optimized value:", result.cost)
print("Optimized parameter:", result.params)
print("Iterations:", result.niter)
print("Cost function calls:", result.funcalls)
print("Gradient function calls:", result.gradcalls)

Optimizer converged
Optimized value: -4.999671477144915
Optimized parameter: [-9.82056843e-03  7.66432752e-02  9.51418965e-02  6.02956014e-02
  3.79603042e-01  2.14981624e-01 -5.64597949e-01  5.70942278e-02
 -8.98413608e-03 -3.02248261e-02  5.99985405e-01  2.86369944e-02
  4.57372527e-01  2.25387953e-01 -5.19614811e-01  3.90692115e-02
  8.53684799e-03 -1.31952094e-02  7.52695346e-01  2.43807022e-02
  5.16032153e-01  2.32264268e-01 -5.05910507e-01  6.06766183e-02
  5.97814126e-03  4.54189863e-03  8.88463907e-01  3.17474563e-02
  5.48131624e-01  2.34734157e-01 -5.32958676e-01  7.15604658e-02
  8.28195567e-04  1.35051470e-02  8.34989893e-01  2.99759243e-02
  5.57991990e-01  2.32341377e-01 -5.56208555e-01  5.17332919e-02
 -1.62171672e-02 -7.99517326e-03  6.39869044e-01  2.35056299e-02
  5.34608165e-01  2.25113227e-01 -5.51124731e-01  2.51094522e-02
 -4.00631092e-02 -5.91952839e-03  4.62880899e-01  3.52301090e-02
  9.34693317e-02  2.13534983e-01  1.06660799e-01  5.00090564e-02]
Iterations: 

## **Reference*:*

TFIM Tutorial: https://phas.ubc.ca/~seme/516/smtutorial2.pdf

# **Applications**

## **1. Quantum Annealing**  
**Mechanism**:  
The TIM Hamiltonian is used as a "driver" in adiabatic quantum computing:  
$$
H(t) = A(t) \sum_n \sigma_n^x + B(t) \sum_n \sigma_n^z \sigma_{n+1}^z
$$  
- Initial state $(t=0)$: Ground state of $(H_{\text{initial}} = -A(0) \sum \sigma_n^x)$ (paramagnetic phase).  
- Final state $(t=T)$: Ground state of $(H_{\text{final}} = -B(T) \sum \sigma_n^z \sigma_{n+1}^z)$ (encodes optimization problem).  

**Example**: Solving the MAX-CUT problem by mapping graph edges to \(J\) couplings.  

## **2. Material Realization ($CoNb_2O_6$)**  
- **Spin Chain Behavior**:  
  Neutron scattering experiments show dispersion relation:  
  $$
  \omega(k) \propto \sqrt{(J - h)^2 + 4Jh \sin^2(k/2)}
  $$  
  Matching TIM predictions.  
- **Critical Field**:  
  At $(h_c = J \approx 1.5 \, \text{T})$, magnetization vanishes, signaling the phase transition.  

## **3. Topological Edge States (Kitaev Chain)**  
**Mapping to Majorana Fermions**:  
Rewrite TIM fermions as Majorana operators:  
$$
\gamma_{2n-1} = a_n + a_n^\dagger, \quad \gamma_{2n} = i(a_n - a_n^\dagger)
$$  
**Edge Modes**:  
For \(J > h\), unpaired Majorana modes $(\gamma_1)$ and $(\gamma_{2N})$ appear at chain ends, leading to degeneracy:  
$$
|\psi_{\text{edge}}\rangle = \frac{1}{\sqrt{2}} (|0\rangle + |1\rangle)
$$  
**Application**: Robust qubits for fault-tolerant quantum computing.  